# All the imports that is used for code

### tensorflow (real time object detection)
### os(path to system files)
### numpy(used for mathematical calculations)
### tarfile(used for compressed file in for of tar)
### zipfile(used for compressed file in zip file format)
### pyttsx3(a module used for conversion of text to speech)

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pyttsx3

### To import all the files from cloned directory of tensorflow available on internet

In [2]:
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


from utils import visualization_utils as vis_util
from utils import label_map_util


## Here the code initializes the engine instance for speech synthesis 
#### Some properties is changed for changing voice from default to selected one

In [3]:
# init function to get an engine instance for the speech synthesis  
engine = pyttsx3.init()


# Set properties _before_ you add things to say
#engine.setProperty('rate', 150)    # Speed percent (can go over 100)
#engine.setProperty('volume', 0.9)  # Volume 0-1
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
#print(voices[0].id) # type of voices
engine.setProperty('voice',voices[1].id)


# Model Prepration
#### SSD with mobilenet model is used available from github which is able to identify nearly 80 objects. Made by Microsoft COCOAPI. It just indicates which model to download actual model is not downloaded by this code

In [4]:
# # Model preparation 
# Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  
# By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.

In [5]:
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.

In [6]:
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

# To download the model specified 

In [7]:
# ## Download Model

if not os.path.exists(MODEL_NAME + '/frozen_inference_graph.pb'):
    print ('Downloading the model')
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())
    print ('Download complete')
else:
    print ('\nModel already exists\n\nWebcam starting...')



Model already exists

Webcam starting...


In [8]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Loading label map
### Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine


In [9]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Actual working of the code
## All the part of detecting objects printing it and converting it into text is done by using above imports

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
engine.say("Welcome to Blind Assistance Recognitor and Visionary")
engine.runAndWait()
# Running the tensorflow session
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
   ret = True
   while (ret):
      ret,image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      #print(classes)
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      #print ([category_index.get(value) for index,value in enumerate(classes[0]) if scores[0,index] > 0.5])

      #print([category_index.get(value).get('name') for index,value in enumerate(classes[0]) if scores[0,index] > 0.5])
#==============================================================================================================================#
# ================================ Code for printing and converting it into text ============================================= #
#==============================================================================================================================#
      name = ([category_index.get(value).get('name') for index,value in enumerate(classes[0]) if scores[0,index] > 0.5])
      print(name)
      engine.say(name)
      engine.runAndWait()
#==============================================================================================================================#

#==============================================================================================================================#
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
#      plt.figure(figsize=IMAGE_SIZE)
#      plt.imshow(image_np)
      cv2.imshow('image',cv2.resize(image_np,(1366,768)))
      #cv2.imshow('image',cv2.resize(image_np,(1024,512))) 
      if cv2.waitKey(25) & 0xFF == ord('q'):
          cv2.destroyAllWindows()
          cap.release()
          print("\nHope You Liked it!")
          engine.say("Hope you liked it sir")
          engine.say("Code by :- Rinkesh, Meet and Dev")
          engine.runAndWait()
          break
#intializing the web camera device


[]
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person']
['person']
['person']
['person', 'person']
['person', 'person', 'cell phone']
['person']
['person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person']
['person', 'person']
['person']
['person', 'person']
['person', 'person']
['person', 'person']
['person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']


Exception ignored in: <function BSTR.__del__ at 0x000001B3153A3AE8>
Traceback (most recent call last):
  File "C:\Users\Rinkesh\Anaconda3\lib\site-packages\comtypes\__init__.py", line 1005, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt


['person', 'person']
['person', 'person']
['person', 'person']
['person', 'person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
['person']
